## Parameters

In [ ]:
bucketPath = "s3://<s3-bucket-name>"

## Read inputs from two paths

In [ ]:
inputPath = bucketPath + "/input/tripdata.csv"
milesDataPath = bucketPath + "/miles-per-rate"

taxiData = spark.read.option("inferSchema", "true").option("header", "true").csv(inputPath)
taxiData.createOrReplaceTempView("taxi_data")

mileRateData = spark.read.parquet(milesDataPath)

## Calculate fare amount by RateCodeId

In [ ]:
from pyspark.sql import functions as F

result = spark.sql("select RatecodeID, sum(fare_amount) from taxi_data group by RatecodeID").toDF("rate_code_id", "total_fare_amount").join(mileRateData, F.col("rate_code") == F.col("rate_code_id")).drop("rate_code")

## Print out the result and write into output path

In [ ]:
resultPath = bucketPath + "/rate-code-status"
result.show()
result.coalesce(1).write.option("header", "true").csv(resultPath)